In [1]:
import numpy as np
import librosa, os, keras, time
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.utils import make_chunks
from collections import deque

Using TensorFlow backend.


In [6]:
dir1 = './Instrument-Classification/IRMAS-TestingData-Part1/Part1/'
dir2 = './Instrument-Classification/IRMAS-TestingData-Part2/Part2/'
dir3 = './Instrument-Classification/IRMAS-TestingData-Part3/Part3/'

#model = keras.models.load_model('ICmodel_pggbvv.h5')
model = keras.models.load_model('ICmodel_PGGVV_CQT_20.h5')

dir_array = [dir1,dir2,dir3]
#instr = ['pia','gac','gel','buz','vio','voi']
instr = ['pia','gac','gel','vio','voi']

In [4]:
def to_categorical(list_final):
    temp_list = [[0]*len(instr)]*(len(list_final))
    for i in range(len(list_final)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list_final[i]): 
                temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list[0]

def prediction(predict):
    prediction = model.predict(predict)
    sum = np.zeros(len(prediction[0]))
    for pred in prediction:
        for i in range(len(pred)):
            sum[i] = sum[i] + pred[i]
    sum[:] = [round(x/len(prediction),2) for x in sum]
    return sum

def average_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)):
        acc = acc + abs(y_pred[i]-y_label[i])
    acc = acc / len(y_pred)
    return round(100*(1-acc),2)

def existance_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)):
        if (y_pred[i]>=0.5 and y_label[i]==1): acc+=1
        #elif (y_pred[i]>=0.5 and y_label[i]==0): acc-=1
        elif (y_pred[i]<0.5 and y_label[i]==0): acc+=1
        #elif (y_pred[i]<0.5 and y_label[i]==1): acc-=1
    acc = acc / len(y_pred)
    return round(100*(acc),2)
    
def predict_songs(start,end):
    sum_acc1 = 0
    sum_acc2 = 0
    it = 1
    songs = 0
    start = 2*start
    end = 2*end - start
    text = True
    unk_instr = False
    for dir in dir_array:
        for f in os.listdir(dir):
    
            if (start!=0):
                start = start -1
                continue
    
            if (unk_instr):
                unk_instr = False
                continue
        
            if (text):
                y_test = []
                x = open(dir+f)
                for line in x.readlines():
                    y_test.append(line[:3])
                    if (not (line[:3] in instr)): 
                        unk_instr = True
                        #print("Unknown instrument in: ",f)
                        #print("")
                if (unk_instr): 
                    continue
                y_label = to_categorical([y_test])
        
            if (not text):
                predict = []
                wav,sr = librosa.load(dir+f,mono=True,sr=22050)
                wav = wav / np.sqrt(np.mean(wav**2))
                chunks = make_chunks(wav,sr)
                for ch in chunks[:-1]:
                    #mel = librosa.feature.melspectrogram(ch,n_mels=96,fmax=11025,n_fft=1024,hop_length=256,power=1)
                    mel = librosa.core.cqt(ch,n_bins=96,hop_length=256)
                    mel = librosa.core.amplitude_to_db(np.abs(mel))
                    predict.append(mel)
                predict = np.expand_dims(np.array(predict),-1)
                y_pred = prediction(predict)
        
                acc1 = average_accuracy(y_pred,y_label)
                acc2 = existance_accuracy(y_pred,y_label)
            
                sum_acc1 = sum_acc1 + acc1
                sum_acc2 = sum_acc2 + acc2
                songs = songs + 1

                print("Predicted track    : ",f)
                print("Expected output    : ", y_label)
                print("Predicted output   : ", y_pred)
                print("Percentage Accuracy: ", acc1,"%")
                print("Existance Accuracy : ", acc2,"%")
                print("")
            
            
            if (it == end): 
                return [sum_acc1,sum_acc2,songs]
            it = it + 1
            text = not text
    return [sum_acc1,sum_acc2,songs]

In [7]:
start = 1
end = 5001
[sum_acc1,sum_acc2,songs] = predict_songs(start,end)

aver_acc1 = round(sum_acc1/songs,2)
aver_acc2 = round(sum_acc2/songs,2)

print("------------------------------------------------------")
print("Average Percentage Accuracy among",songs,"songs: ",aver_acc1,"%")
print("Average Existance  Accuracy among",songs,"songs: ",aver_acc2,"%")

Predicted track    :  (02) dont kill the whale-11.wav
Expected output    :  [0, 0, 1, 0, 0]
Predicted output   :  [0.23 0.01 0.74 0.14 0.62]
Percentage Accuracy:  74.8 %
Existance Accuracy :  80.0 %

Predicted track    :  (02) dont kill the whale-12.wav
Expected output    :  [0, 0, 1, 0, 1]
Predicted output   :  [0.25 0.01 0.84 0.07 0.79]
Percentage Accuracy:  86.0 %
Existance Accuracy :  100.0 %

Predicted track    :  (02) dont kill the whale-13.wav
Expected output    :  [0, 0, 1, 0, 1]
Predicted output   :  [0.17 0.   0.76 0.07 0.79]
Percentage Accuracy:  86.2 %
Existance Accuracy :  100.0 %

Predicted track    :  (02) dont kill the whale-14.wav
Expected output    :  [0, 0, 1, 0, 1]
Predicted output   :  [0.15 0.   0.82 0.14 0.67]
Percentage Accuracy:  84.0 %
Existance Accuracy :  100.0 %

Predicted track    :  (02) dont kill the whale-15.wav
Expected output    :  [1, 0, 1, 0, 0]
Predicted output   :  [0.36 0.   0.88 0.23 0.41]
Percentage Accuracy:  72.0 %
Existance Accuracy :  80.0 

Predicted track    :  01 - Canto das tre¦Γs rac¦πas-12.wav
Expected output    :  [0, 1, 0, 0, 1]
Predicted output   :  [0.12 0.24 0.13 0.08 0.92]
Percentage Accuracy:  76.6 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Canto das tre¦Γs rac¦πas-13.wav
Expected output    :  [0, 1, 0, 0, 1]
Predicted output   :  [0.14 0.18 0.09 0.18 0.91]
Percentage Accuracy:  73.6 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Canto das tre¦Γs rac¦πas-14.wav
Expected output    :  [0, 1, 0, 0, 1]
Predicted output   :  [0.22 0.32 0.08 0.09 0.89]
Percentage Accuracy:  76.4 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Canto das tre¦Γs rac¦πas-2.wav
Expected output    :  [0, 1, 0, 0, 1]
Predicted output   :  [0.05 0.52 0.02 0.04 0.88]
Percentage Accuracy:  85.8 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Canto das tre¦Γs rac¦πas-3.wav
Expected output    :  [0, 1, 0, 0, 1]
Predicted output   :  [0.07 0.64 0.01 0.11 0.82]
Percentage Accuracy:  85.4 %
Exista

Predicted track    :  01 - Honky Cat-5.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.29 0.13 0.21 0.21 0.49]
Percentage Accuracy:  64.6 %
Existance Accuracy :  60.0 %

Predicted track    :  01 - Honky Cat-6.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.23 0.08 0.02 0.21 0.84]
Percentage Accuracy:  75.2 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Honky Cat-7.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.21 0.1  0.19 0.31 0.69]
Percentage Accuracy:  66.0 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Honky Cat-8.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.42 0.33 0.02 0.16 0.63]
Percentage Accuracy:  70.8 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Inolvidable-1.wav
Expected output    :  [1, 0, 0, 0, 0]
Predicted output   :  [0.68 0.1  0.01 0.16 0.01]
Percentage Accuracy:  88.0 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Inolvidable-10.wav
Expect

Predicted track    :  01 - Orchids in the Moonlight-5.wav
Expected output    :  [0, 1, 0, 0, 0]
Predicted output   :  [0.05 0.8  0.11 0.45 0.  ]
Percentage Accuracy:  83.8 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Orchids in the Moonlight-6.wav
Expected output    :  [0, 1, 0, 0, 0]
Predicted output   :  [0.16 0.82 0.04 0.16 0.03]
Percentage Accuracy:  88.6 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Orchids in the Moonlight-7.wav
Expected output    :  [0, 1, 0, 0, 0]
Predicted output   :  [0.4  0.9  0.08 0.14 0.02]
Percentage Accuracy:  85.2 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Orchids in the Moonlight-8.wav
Expected output    :  [0, 1, 0, 0, 0]
Predicted output   :  [0.4  0.84 0.04 0.14 0.04]
Percentage Accuracy:  84.4 %
Existance Accuracy :  100.0 %

Predicted track    :  01 - Orchids in the Moonlight-9.wav
Expected output    :  [0, 1, 0, 0, 0]
Predicted output   :  [0.53 0.94 0.05 0.15 0.01]
Percentage Accuracy:  84.0 %
Exista

Predicted track    :  01 - So What-4.wav
Expected output    :  [1, 0, 0, 0, 0]
Predicted output   :  [0.6  0.12 0.01 0.56 0.05]
Percentage Accuracy:  77.2 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - So What-5.wav
Expected output    :  [1, 0, 0, 0, 0]
Predicted output   :  [0.62 0.06 0.01 0.52 0.08]
Percentage Accuracy:  79.0 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Wee Baby Blues-11.wav
Expected output    :  [1, 0, 1, 0, 1]
Predicted output   :  [0.62 0.02 0.28 0.09 0.6 ]
Percentage Accuracy:  67.8 %
Existance Accuracy :  80.0 %

Predicted track    :  01 - Wee Baby Blues-2.wav
Expected output    :  [1, 0, 1, 0, 1]
Predicted output   :  [0.65 0.05 0.16 0.19 0.48]
Percentage Accuracy:  61.0 %
Existance Accuracy :  60.0 %

Predicted track    :  01 - Wee Baby Blues-3.wav
Expected output    :  [1, 0, 1, 0, 1]
Predicted output   :  [0.54 0.06 0.45 0.28 0.45]
Percentage Accuracy:  62.0 %
Existance Accuracy :  60.0 %

Predicted track    :  01 - Wee Baby Blues-4

...

Predicted track    :  06 - the human league - reproduction - empire state human-3.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.33 0.   0.06 0.34 0.71]
Percentage Accuracy:  79.6 %
Existance Accuracy :  100.0 %

Predicted track    :  06 - the human league - reproduction - empire state human-4.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.23 0.   0.2  0.58 0.59]
Percentage Accuracy:  71.6 %
Existance Accuracy :  80.0 %

Predicted track    :  06 - the human league - reproduction - empire state human-5.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.34 0.   0.06 0.29 0.87]
Percentage Accuracy:  83.6 %
Existance Accuracy :  100.0 %

Predicted track    :  06 - the human league - reproduction - empire state human-6.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.37 0.   0.08 0.31 0.73]
Percentage Accuracy:  79.4 %
Existance Accuracy :  100.0 %

Predicted track    :  06 - the human league - reproduction - empire s

Predicted track    :  07 palestrina - stabat mater-15.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.21 0.   0.   0.64 0.86]
Percentage Accuracy:  80.2 %
Existance Accuracy :  80.0 %

Predicted track    :  07 palestrina - stabat mater-16.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.33 0.   0.01 0.56 0.84]
Percentage Accuracy:  78.8 %
Existance Accuracy :  80.0 %

Predicted track    :  07 palestrina - stabat mater-17.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.34 0.   0.02 0.69 0.91]
Percentage Accuracy:  77.2 %
Existance Accuracy :  80.0 %

Predicted track    :  07 palestrina - stabat mater-18.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.14 0.   0.   0.57 0.66]
Percentage Accuracy:  79.0 %
Existance Accuracy :  80.0 %

Predicted track    :  07 palestrina - stabat mater-19.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.18 0.   0.02 0.51 0.68]
Percentage Accuracy:  79.4 %
Existance 

Predicted track    :  08 - Rufus Wainwright - Poses-19.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.35 0.04 0.14 0.32 0.63]
Percentage Accuracy:  69.6 %
Existance Accuracy :  80.0 %

Predicted track    :  08 - Rufus Wainwright - Poses-20.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.42 0.04 0.18 0.21 0.77]
Percentage Accuracy:  75.2 %
Existance Accuracy :  80.0 %

Predicted track    :  08 - Rufus Wainwright - Poses-21.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.51 0.17 0.01 0.16 0.65]
Percentage Accuracy:  76.4 %
Existance Accuracy :  100.0 %

Predicted track    :  08 - Rufus Wainwright - Poses-23.wav
Expected output    :  [1, 0, 0, 0, 0]
Predicted output   :  [0.96 0.28 0.01 0.29 0.01]
Percentage Accuracy:  87.4 %
Existance Accuracy :  100.0 %

Predicted track    :  08 - Rufus Wainwright - Poses-3.wav
Expected output    :  [1, 0, 0, 0, 1]
Predicted output   :  [0.7  0.22 0.   0.03 0.93]
Percentage Accuracy:  87.6 %
Exis

Predicted track    :  08-Concerto in A major, K 219 -2- Adagio-44.wav
Expected output    :  [0, 0, 0, 1, 0]
Predicted output   :  [0.01 0.   0.   0.97 0.02]
Percentage Accuracy:  98.8 %
Existance Accuracy :  100.0 %

Predicted track    :  08-Concerto in A major, K 219 -2- Adagio-45.wav
Expected output    :  [0, 0, 0, 1, 0]
Predicted output   :  [0.01 0.01 0.   0.94 0.02]
Percentage Accuracy:  98.0 %
Existance Accuracy :  100.0 %

Predicted track    :  08-Concerto in A major, K 219 -2- Adagio-46.wav
Expected output    :  [0, 0, 0, 1, 0]
Predicted output   :  [0.02 0.01 0.   0.97 0.06]
Percentage Accuracy:  97.6 %
Existance Accuracy :  100.0 %

Predicted track    :  08-Concerto in A major, K 219 -2- Adagio-9.wav
Expected output    :  [0, 0, 0, 1, 0]
Predicted output   :  [0.   0.   0.   0.99 0.02]
Percentage Accuracy:  99.4 %
Existance Accuracy :  100.0 %

Predicted track    :  10 - Johann Sebastian Bach - English Suite No. 3 in G major, BWV 808 (Andra¦Αs Schiff) (1)-1.wav
Expected outpu

Predicted track    :  13 Chet Atkins & Jerry Reed - Sneakin' Around-9.wav
Expected output    :  [0, 0, 1, 0, 0]
Predicted output   :  [0.2  0.29 0.65 0.22 0.08]
Percentage Accuracy:  77.2 %
Existance Accuracy :  100.0 %

Predicted track    :  13 Living in the Past-3.wav
Expected output    :  [0, 0, 1, 1, 1]
Predicted output   :  [0.05 0.16 0.2  0.17 0.86]
Percentage Accuracy:  60.4 %
Existance Accuracy :  60.0 %

Predicted track    :  13 Living in the Past-7.wav
Expected output    :  [0, 0, 1, 1, 1]
Predicted output   :  [0.06 0.18 0.22 0.08 0.84]
Percentage Accuracy:  58.0 %
Existance Accuracy :  60.0 %

Predicted track    :  14 - Tomorrow Never Knows-11.wav
Expected output    :  [1, 0, 0, 0, 0]
Predicted output   :  [0.35 0.02 0.63 0.55 0.14]
Percentage Accuracy:  60.2 %
Existance Accuracy :  40.0 %

Predicted track    :  14 - Tomorrow Never Knows-2.wav
Expected output    :  [0, 0, 0, 0, 1]
Predicted output   :  [0.23 0.02 0.51 0.19 0.57]
Percentage Accuracy:  72.4 %
Existance Accura